# Data tiling 1

## import libraries

In [1]:
# from rdflib import Graph, URIRef, Literal, RDF
import open3d as o3d
import os
from pathlib import Path
import numpy as np
import laspy
import cv2
import copy
import matplotlib.pyplot as plt
import pye57 


from context import geomapi #! only use this if you are inside geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from geomapi.utils import imageutils as iu
from geomapi.tools import progresstools as pt
from  geomapi.utils import cadutils as cadu
from geomapi.tools import validationtools as vt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
cadu.

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## input

In [4]:
e57_path='D:/Gerardo-benchmark/Week22/week 22 lidar.e57' #use forward slashing otherwise the spaces cause erors
e57_path='D:/Gerardo-benchmark/Week22/week22_sfm.e57' #use forward slashing otherwise the spaces cause erors
e57_path='C:/Users/u0094523/Documents/week 22/week 22 lidar.e57'
output_dir='C:/Users/u0094523/Documents/week 22/'
tile_size=20#m
percentage=1

## import data

In [29]:

e57 = pye57.E57(e57_path)
gmu.e57_update_point_field(e57)
pointCloudNodes=[]
for s in np.arange(1):
    resource=gmu.e57_to_pcd(e57,e57Index=0,percentage=percentage)
    node=PointCloudNode(resource=resource,
                        path=e57_path,
                        e57Index=s,
                        percentage=percentage)
    node.pointCount=len(resource.points)
    pointCloudNodes.append(node)
print(f'{len(pointCloudNodes)} nodes created!')

1 nodes created!


In [28]:
header = e57.get_header(0)
print(header.pretty_print())
# header.point_fields

["<StringNode 'guid'>: {149e5ca1_1cc1_4214_bc97_83d6b35f9eb8}", "<StringNode 'name'>: academiestraat week 22b 1", "<StringNode 'description'>: this file is generated by Autodesk ReCap", "<StructureNode 'indexBounds'>", "    <IntegerNode 'rowMinimum'>: 0", "    <IntegerNode 'rowMaximum'>: 2446", "    <IntegerNode 'columnMinimum'>: 0", "    <IntegerNode 'columnMaximum'>: 4893", "    <IntegerNode 'returnMinimum'>: 0", "    <IntegerNode 'returnMaximum'>: 0", "<StructureNode 'intensityLimits'>", "    <IntegerNode 'intensityMaximum'>: 65535", "    <IntegerNode 'intensityMinimum'>: 0", "<StructureNode 'colorLimits'>", "    <IntegerNode 'colorRedMaximum'>: 255", "    <IntegerNode 'colorRedMinimum'>: 0", "    <IntegerNode 'colorGreenMaximum'>: 255", "    <IntegerNode 'colorGreenMinimum'>: 0", "    <IntegerNode 'colorBlueMaximum'>: 255", "    <IntegerNode 'colorBlueMinimum'>: 0", "<StructureNode 'sphericalBounds'>", "    <FloatNode 'rangeMinimum'>: 0.0", "    <FloatNode 'rangeMaximum'>: 42949.67

In [40]:
pointCloudNodes=tl.e57path_to_nodes_mutiprocessing(e57_path) # nodes
print(f'{len(pointCloudNodes)} nodes created!')

2 nodes created!


In [11]:
print(pointCloudNodes[0].resource) #this is the open3d point cloud


PointCloud with 11975618 points.


In [41]:
#visualize your data (meshes, lines or point clouds)
o3d.visualization.draw_geometries([pointCloudNodes[0].resource]) # use resource, not Node itself 

## merge point clouds

In [42]:
joined_pcd=gmu.join_geometries([n.resource for n in pointCloudNodes])
# pointCloudNode=PointCloudNode(resource= joined_pcd) # new node automatically computes box and stuff

In [44]:
o3d.visualization.draw_geometries([joined_pcd]) # use resource, not Node itself 

In [38]:
#release memory
for n in pointCloudNodes:
    del n.resource
    # n.resource =None

## tile the data

In [43]:
print(len(np.asarray(joined_pcd.points)))

23990420


In [36]:
# use AxisAlignedBoundingBox or OrientedBoundingBox 
box=joined_pcd.get_axis_aligned_bounding_box()

In [45]:
#subdivide box

idxLists,pathLists=pt.subdivide_pcd_per_box(joined_pcd,[tile_size,tile_size,100])

In [43]:
#select points
for i,name in zip(idxLists,pathLists):
    pcd=joined_pcd.select_by_index(i)

    #write it to drive
    o3d.io.write_point_cloud(os.path.join(output_dir,name+'.pcd'), pcd)

[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.
[Open3D WARNING] Write PCD failed: unable to generate header.


In [46]:
pcds=[]
for i,name in zip(idxLists,pathLists):
    pcd=joined_pcd.select_by_index(i)
    pcds.append(pcd)

In [47]:
joined_pcd2=gmu.join_geometries([n for n in pcds])
o3d.visualization.draw_geometries([joined_pcd2]) # use resource, not Node itself 